In [1]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
seasons = list(range(2016,2023))

In [3]:
data_dir = "data"
standings_dir = os.path.join(data_dir, "standings")
scores_dir = os.path.join(data_dir, "scores")

In [4]:
def get_html(url, selector, sleep=2, retries=3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)

        try:
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            print(driver.title)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            html = soup.select_one(selector)
            driver.quit()
        except TimeoutException:
            print(f"Timeout error on {url}")
            driver.quit()
            continue
        else:
            break

    return html

In [5]:
def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = get_html(url, '.filter')
    links = html.find_all("a")
    href = [l['href'] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(standings_dir, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(str(html))

In [7]:
for season in seasons:
    scrape_season(season)

C:\Users\gonzalo.velazquez\AppData\Local\Temp\ipykernel_121948\3475900239.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com


KeyboardInterrupt: 

In [6]:
standings_files = os.listdir(standings_dir)

In [7]:
def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(scores_dir, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = get_html(url, "#content")

        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(str(html))

In [8]:
for f in standings_files:
    filepath = os.path.join(standings_dir, f)
    scrape_game(filepath)

C:\Users\gonzalo.velazquez\AppData\Local\Temp\ipykernel_225040\3475900239.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
